## DRF Access Policy

`drf-access-policy` provides a way to define complex access rules for your Django Rest Framework APIs in a declarative, readable format. It is inspired by AWS IAM policies, allowing you to separate your permission logic from your views and models. This approach is significantly more maintainable than writing complex `if/else` chains in standard DRF permission classes.

### Why use it?
1.  **Declarative Syntax**: Permissions are defined as a list of statements, making them easy to read and audit.
2.  **Separation of Concerns**: Keeps your ViewSets clean by moving permission logic to its own class.
3.  **Attribute-Based Access Control (ABAC)**: Easily write rules based on object attributes (e.g., "User can only edit if they are the owner" or "User can only publish if the article is in draft state").

### Core Concepts
An `AccessPolicy` is a class that defines a `statements` list. Each statement rules on whether a request should be allowed or denied.

- **action**: The ViewSet method(s) this rule applies to (e.g., `list`, `retrieve`, `create`, or custom actions).
- **principal**: The user(s) this rule applies to (e.g., `*` for anyone, `authenticated` for logged-in users, `group:admin` for specific groups).
- **effect**: `allow` or `deny`.
- **condition** (optional): A list of custom methods to check for finer-grained control (e.g., `is_owner`).

### Installation & Setup

Use `pipenv` (or `pip`) to install the package:

```bash
pipenv install drf-access-policy
```

No specific `INSTALLED_APPS` configuration is required for `drf-access-policy` itself, as it is a utility library.

### Basic Usage Structure

To use it, you create a policy class inheriting from `AccessPolicy` and assign it to your ViewSet's `permission_classes`.

#### 1. Define the Policy
```python
from rest_framework_access_policy import AccessPolicy

class ArticleAccessPolicy(AccessPolicy):
    statements = [
        {
            "action": ["list", "retrieve"],
            "principal": "*",
            "effect": "allow"
        },
        {
            "action": ["create", "update"],
            "principal": ["authenticated"],
            "effect": "allow"
        },
        {
            "action": ["destroy"],
            "principal": ["group:admin"],
            "effect": "allow"
        }
    ]
```

#### 2. Apply to ViewSet
```python
from rest_framework import viewsets
from .models import Article
from .serializers import ArticleSerializer

class ArticleViewSet(viewsets.ModelViewSet):
    queryset = Article.objects.all()
    serializer_class = ArticleSerializer
    permission_classes = (ArticleAccessPolicy,)
```

### Custom Conditions

Conditions are where `drf-access-policy` truly shines. You can define methods starting with `is_` (or others if configured) to check dynamic attributes.

For example, ensuring a user can only edit their own articles:

```python
class ArticleAccessPolicy(AccessPolicy):
    statements = [
        {
            "action": ["update", "partial_update"],
            "principal": ["authenticated"],
            "effect": "allow",
            "condition": ["is_owner"]
        },
        # ... other rules
    ]

    def is_owner(self, request, view, action) -> bool:
        # Retrieve the specific object being accessed
        # Note: view.get_object() triggers a database query.
        article = view.get_object()
        return request.user == article.author
```

You can chain multiple conditions. All conditions in the list must return `True` for the statement to match.

### Principals

The `principal` key defines *who* the rule targets. Some meaningful values include:

- `*`: Any user (including anonymous).
- `authenticated`: Any logged-in user.
- `anonymous`: Any user who is NOT logged in.
- `admin`: Django superusers.
- `staff`: Django staff users.
- `group:name`: Users belonging to the Django group named "name".
- `id:123`: User with specific ID (rarely used, but possible).

### Scoping QuerySets

Access policies can also easily filter the `get_queryset` method of your ViewSet to automatically restrict the data returned to the user, based on the same rules.

```python
class ArticleAccessPolicy(AccessPolicy):
    # ... statements ...

    @classmethod
    def scope_queryset(cls, request, queryset):
        if request.user.is_staff:
            return queryset
        return queryset.filter(author=request.user)

# In ViewSet:
class ArticleViewSet(viewsets.ModelViewSet):
    permission_classes = (ArticleAccessPolicy,)
    
    def get_queryset(self):
        return ArticleAccessPolicy.scope_queryset(
            self.request, Article.objects.all()
        )
```

### Best Practices

1.  **Deny by Default**: If no statement matches, AccessPolicy denies access by default. Rely on this safety net.
2.  **Order Matters**: Statements are evaluated in order. The first one that matches `action` and `principal` (and `condition`) determines the outcome. 
3.  **Keep Conditions Lightweight**: Remember that conditions stick to the request lifecycle. Avoid expensive operations if possible, or memoize them.
4.  **Use `scope_queryset`**: Whenever possible, filter data at the database level using scope_queryset rather than fetching everything and permission-checking one by one.